In [44]:
from tqdm import tqdm
import cv2
import multiprocessing as mp
import numpy as np
import os
from pathlib import Path
import pandas as pd
from typing import Optional

In [114]:
from utils.utils_IO import combine_images, sorted_alphanumeric # write_video

In [115]:
# TODO: should be made a util. consider wrapping all funcs in a class
def extract_frames(indices, video_path, exact=True):
    assert(os.path.isfile(video_path))
    cap = cv2.VideoCapture(video_path)
    totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    frames = []
    if exact:
        indices_dict = dict((idx, 0) for idx in indices)
        curr_idx = 0
        while len(indices_dict.keys()) > 0 and curr_idx < totalFrames:
            ret, frame = cap.read()
            if curr_idx in indices_dict:
                frames.append(frame)
                del indices_dict[curr_idx]
            curr_idx += 1

    else:
        for frame_idx in indices:
            # get total number of frames
            # check for valid frame number
            if frame_idx >= 0 & frame_idx <= totalFrames:
                # set frame position
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = cap.read()
                frames.append(frame)
    return frames

In [195]:
def write_video(
    frames: Optional[list] = None, 
    image_dir: Optional[str] = None, 
    out_file: Optional[str] = "./out.mp4", 
    fps: Optional[int] = 5, 
    add_frame_num: Optional[bool] = False, 
    added_text: Optional[list] = None,
):
    img_array = []

    if frames is not None:
        im_list = frames
        for img in im_list:
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)

    if image_dir is not None:
        im_list = os.listdir(image_dir)
        im_list = sorted_alphanumeric(im_list)

        if ".DS_Store" in im_list:
            im_list.remove(".DS_Store")

        for filename in im_list:
            img = cv2.imread(os.path.join(image_dir, filename))
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)

    out = cv2.VideoWriter(
        filename=out_file,
        fourcc=cv2.VideoWriter_fourcc("m", "p", "4", "v"),
        fps=fps,
        frameSize=size,
    )

    cv2.VideoWriter()
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    if added_text is not None:
        assert(len(added_text) == len(img_array))

    for i in range(len(img_array)):
        if add_frame_num:
            cv2.putText(
                img_array[i], f"{i}", (10, 30), font, 1, (0, 0, 255), 1, cv2.LINE_AA
            )
        if added_text is not None:
            cv2.putText(
                img_array[i], added_text[i], (0,round(img_array[i].shape[0]*.9)), font, .5, (255, 255, 255))
        out.write(img_array[i])
    out.release()

In [326]:
general_video_path = '/Volumes/sawtell-locker/C1/free/vids'
assert(os.path.isdir(general_video_path))

In [327]:
# csv_file = '/Users/danbiderman/Dropbox/Columbia/1.Dan/Research/Paninski/Video_Datasets/Sawtell-data/C1_prey_capture.csv'
# assert(os.path.isfile(csv_file))
csv_path = '/Volumes/sawtell-locker/C1/free/C1_prey_capture_events.csv'
assert(os.path.isfile(csv_path))

In [328]:
df = pd.read_csv(csv_path, header=0)

In [329]:
df.head()

,Path,Date,Fish,Reproj Vid Frame START,Reproj Vid Frame STOP,View(s),Error,User,"Notes (capture not properly tracked, false positives, etc.)",Original Vid Frame START,Original Vid Frame STOP
0,Z:\C1\free\3d_reconstruction\V2\,20201218,Neil,115.0,128.0,NaN,NaN,DN,NaN,1150,1280
1,Z:\C1\free\3d_reconstruction\V2\,20201218,Neil,555.0,569.0,NaN,NaN,DN,NaN,5550,5690
2,Z:\C1\free\3d_reconstruction\V2\,20201218,Neil,1795.0,1818.0,NaN,NaN,DN,NaN,17950,18180
3,Z:\C1\free\3d_reconstruction\V2\,20201218,Neil,2560.0,2568.0,NaN,NaN,DN,NaN,25600,25680
4,Z:\C1\free\3d_reconstruction\V2\,20201218,Neil,2800.0,2819.0,NaN,NaN,DN,NaN,28000,28190


In [331]:
def get_vid_folder_name(general_video_path: str, date: str, name: str) -> str:
    '''return a path for videos folder from names '''
    full_path = os.path.join(general_video_path , date + '_' + name)
    assert(os.path.isdir(full_path))
    return full_path

In [57]:
DN_data = df.loc[df['User'] == 'DN']

In [59]:
DN_data.shape

(75, 11)

In [64]:
merged = str(DN_data['Date']) + DN_data['Fish']
print(merged)
#print(DN_data['Date'].unique())
#print(DN_data['Fish'].unique())

0     0     20201218\n1     20201218\n2     20201218...
1     0     20201218\n1     20201218\n2     20201218...
2     0     20201218\n1     20201218\n2     20201218...
3     0     20201218\n1     20201218\n2     20201218...
4     0     20201218\n1     20201218\n2     20201218...
                            ...                        
70    0     20201218\n1     20201218\n2     20201218...
71    0     20201218\n1     20201218\n2     20201218...
72    0     20201218\n1     20201218\n2     20201218...
73    0     20201218\n1     20201218\n2     20201218...
74    0     20201218\n1     20201218\n2     20201218...
Name: Fish, Length: 75, dtype: object


In [227]:
list_concat = []
for i in range(DN_data.shape[0]): # loop over rows and merge date and fish strings
    #list_concat.append(str(DN_data['Date'][i]) + '_' + DN_data['Fish'][i])
    list_concat.append(get_vid_folder_name(general_video_path, 
                            str(DN_data["Date"][i]), 
                                DN_data["Fish"][i]))
unique_list = list(set(list_concat))

In [228]:
print(unique_list)
# want: check if unique. if so, join all the frames from that video as one video.

['/Volumes/sawtell-locker/C1/free/vids/20201218_Neil', '/Volumes/sawtell-locker/C1/free/vids/20201120_Greg', '/Volumes/sawtell-locker/C1/free/vids/20201114_Greg', '/Volumes/sawtell-locker/C1/free/vids/20201224_Neil']


In [305]:
cond_list = ['Neil PRE C1 lesion', 'Greg POST C1 sham', 'Greg PRE C1 sham', 'Neil POST C1 lesion' ]
cond_list

['Neil PRE C1 lesion',
 'Greg POST C1 sham',
 'Greg PRE C1 sham',
 'Neil POST C1 lesion']

take all frames from a given session and concat them into one long video
* loop over the unique video paths
* get the desired frames
* make a video and save in a separate folder name -- do not over write

In [319]:
# create a list of lists
frame_nums = list(range(len(unique_list)))
for i in range(len(frame_nums)):
    frame_nums[i] = []

In [320]:
frame_pad = 200
for j in range(DN_data.shape[0]):
    folder_name = get_vid_folder_name(general_video_path, 
                            str(DN_data["Date"][j]), 
                                DN_data["Fish"][j])
    video_path = os.path.join(folder_name,
                              'concatenated_tracking.avi')
    curr_sess = folder_name.split('/')[-1]
    ind = np.where(np.asarray([ud in [folder_name] for ud in unique_dict]))[0][0]
    frame_nums[ind].append(np.arange(DN_data["Original Vid Frame START"][j]-frame_pad, 
                       DN_data["Original Vid Frame STOP"][j]+frame_pad))

In [321]:
for i in range(len(frame_nums)):
    frame_nums[i] = np.concatenate(frame_nums[i])

In [309]:
os.cpu_count()

8

In [322]:
for i, path in enumerate(unique_list):
    print(i, path)
    video_path = os.path.join(path,'concatenated_tracking.avi' ) # change to concatenated if don't want the tracking
    frames = extract_frames(indices = frame_nums[i], video_path=video_path, exact=True)
    added_text = list(np.repeat(cond_list[i], len(frames)))
    write_video(
        frames=frames,
        out_file= os.path.join(path, 'prey_capture_highlights.mov'),
        fps=100, # original is 100 fps frame rate
        add_frame_num=False,
        added_text=added_text)

0 /Volumes/sawtell-locker/C1/free/vids/20201218_Neil
1 /Volumes/sawtell-locker/C1/free/vids/20201120_Greg
2 /Volumes/sawtell-locker/C1/free/vids/20201114_Greg
3 /Volumes/sawtell-locker/C1/free/vids/20201224_Neil


## concat two vids
first create a file with a list of videos:
`(echo file '/Volumes/sawtell-locker/C1/free/vids/20201218_Neil/prey_capture_highlights.mov' & echo file '/Volumes/sawtell-locker/C1/free/vids/20201224_Neil/prey_capture_highlights.mov' )>list.txt`

then concat and save

`ffmpeg -safe 0 -f concat -i list.txt -c copy output.mp4`

In [303]:
# # that works
# os.system('(echo file \'/Volumes/sawtell-locker/C1/free/vids/20201218_Neil/prey_capture_highlights.mov\' & echo file \'/Volumes/sawtell-locker/C1/free/vids/20201224_Neil/prey_capture_highlights.mov\' )>list.txt')
# status = os.system('/Users/danbiderman/miniconda3/envs/3d-pose/bin/ffmpeg -safe 0 -f concat -i list.txt -c copy {}_pre_post.mp4'.format('Neil'))
# print(status)

In [300]:
def write_file_with_paths(paths: list, txt_filename: str):
    f = open(txt_filename,"w+")
    for p in paths:
         f.write('file ' + "%s\r\n" % p)
    f.close() 

In [299]:
def concat_videos(ffmpeg_path: str, txt_filename: str, out_file_name: str = 'concat', out_format: str = '.mov'):
    command = ffmpeg_path + ' ' + '-safe 0 -f concat -i' + ' ' + txt_filename + ' ' + '-c copy' + ' ' + out_file_name + out_format
    status = os.system(command)
    if status==0:
        print('Concat video saved as' + ' ' + out_file_name + out_format)
    else:
        print('Concat failed, error number %i. \n Check that the .txt file exists and that there is no existing video called %s.' % (status, out_file_name+out_format))
    return command, status

In [301]:
def make_concat_video(paths: list, 
                      txt_filename: str, 
                      ffmpeg_path: Optional[str] = '/Users/danbiderman/miniconda3/envs/3d-pose/bin/ffmpeg', 
                      out_file_name: Optional[str] = 'concat', 
                      out_format: Optional[str] = '.mov'):
    write_file_with_paths(paths, txt_filename)
    assert(os.path.isfile(txt_filename))
    command, status = concat_videos(ffmpeg_path, txt_filename, out_file_name, out_format)
    return command, status

In [323]:
paths_Neil = ['\'/Volumes/sawtell-locker/C1/free/vids/20201218_Neil/prey_capture_highlights.mov\'',
        '\'/Volumes/sawtell-locker/C1/free/vids/20201224_Neil/prey_capture_highlights.mov\'']
paths_Greg = ['\'/Volumes/sawtell-locker/C1/free/vids/20201114_Greg/prey_capture_highlights.mov\'',
        '\'/Volumes/sawtell-locker/C1/free/vids/20201120_Greg/prey_capture_highlights.mov\'']

In [325]:
command, status = make_concat_video(paths=paths_Neil, 
                                    txt_filename='list.txt', 
                                    out_file_name= '{}_pre_post'.format('Neil'),
                                   out_format = '.mov')
command, status = make_concat_video(paths=paths_Greg, 
                                    txt_filename='list.txt', 
                                    out_file_name= '{}_pre_post'.format('Greg'),
                                   out_format = '.mov')

Concat video saved as Neil_pre_post.mov
Concat video saved as Greg_pre_post.mov


In [291]:
command, status = concat_videos('/Users/danbiderman/miniconda3/envs/3d-pose/bin/ffmpeg', 
              'list.txt',
             '{}_pre_post'.format('Neil'),
             '.mp4')
#command == '/Users/danbiderman/miniconda3/envs/3d-pose/bin/ffmpeg -safe 0 -f concat -i list.txt -c copy {}_pre_post.mp4'.format('Neil')

Concat failed, error number 256. 
 Check that the .txt file exists and that there is no existing video called Neil_pre_post.mp4.


In [ ]:
/Users/danbiderman/miniconda3/envs/3d-pose/bin/ffmpeg -safe 0 -f concat -i list.txt -c copy Neil_pre_post.mp4
